In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
kickoff_url = "https://www.vlr.gg/event/matches/1923/champions-tour-2024-americas-kickoff/?series_id=all"
stage1_url = "https://www.vlr.gg/event/matches/2004/champions-tour-2024-americas-stage-1/?series_id=all"
stage2_url =  "https://www.vlr.gg/event/matches/2095/champions-tour-2024-americas-stage-2/?series_id=4031" 

In [5]:
urls = [kickoff_url, stage1_url, stage2_url ] 
match_links =[]

In [6]:
#get kickoff links 
for url in urls:
    data = requests.get(url)
    soup = BeautifulSoup(data.text) 
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [str(l) for l in links]
    links = [l for l in links if 'vs' in l]
    links = [f"https://www.vlr.gg{l}" for l in links]
    [match_links.append(link) for link in links]
match_links

['https://www.vlr.gg/295605/nrg-esports-vs-furia-champions-tour-2024-americas-kickoff-opening-a',
 'https://www.vlr.gg/295606/cloud9-vs-mibr-champions-tour-2024-americas-kickoff-opening-a',
 'https://www.vlr.gg/295610/loud-vs-sentinels-champions-tour-2024-americas-kickoff-opening-b',
 'https://www.vlr.gg/295611/leviat-n-vs-100-thieves-champions-tour-2024-americas-kickoff-opening-b',
 'https://www.vlr.gg/295615/kr-esports-vs-g2-esports-champions-tour-2024-americas-kickoff-opening-c',
 'https://www.vlr.gg/295608/furia-vs-mibr-champions-tour-2024-americas-kickoff-elim-a',
 'https://www.vlr.gg/295613/sentinels-vs-100-thieves-champions-tour-2024-americas-kickoff-elim-b',
 'https://www.vlr.gg/295607/nrg-esports-vs-cloud9-champions-tour-2024-americas-kickoff-winners-a',
 'https://www.vlr.gg/295612/loud-vs-leviat-n-champions-tour-2024-americas-kickoff-winners-b',
 'https://www.vlr.gg/295617/g2-esports-vs-evil-geniuses-champions-tour-2024-americas-kickoff-winners-c',
 'https://www.vlr.gg/295609

In [7]:
#get stage 1 links
#data = requests.get(stage1_url)
#soup = BeautifulSoup(data.text) 
#links = soup.find_all('a')
#links = [l.get("href") for l in links]
#links = [str(l) for l in links]
#links = [l for l in links if 'vs' in l]
#stage1_urls = [f"https://www.vlr.gg{l}" for l in links]
#stage1_urls

In [8]:
#links = [] 
#for link in kickoff_urls: 
   # links.append(link)
#for link in stage1_urls: 
  #  links.append(link) 
#links

In [9]:
#creating methods needed to fill tables 
def determine_winner(ct, t, ot):
    team1_total = ct[0] + t[0] + ot[0]
    team2_total = ct[1] + t[1] + ot[1]
    if team1_total > team2_total: 
        return 0
    else: 
        return 1 

def get_maps(match_soup): 
    links = match_soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [str(l) for l in links]
    links = [l for l in links if 'overview' in l]
    link = f"https://www.vlr.gg{links[0]}"
    data = requests.get(link)
    soup = BeautifulSoup(data.text)
    maps = soup.select('div.vm-stats-game-header')
    return maps
def get_teams(map): 
     teams = map.select('div.team-name')
     teams = [n.get_text().strip() for n in teams]
     return teams 
def get_ct(map): 
    ct_rounds = map.select('span.mod-ct')
    ct_rounds = [ct_rounds[0], ct_rounds[1]]
    ct_rounds = [int(n.get_text().strip()) for n in ct_rounds]
    return ct_rounds
def get_t(map): 
    t_rounds = map.select('span.mod-t')
    t_rounds = [t_rounds[0], t_rounds[1]]
    t_rounds = [int(n.get_text().strip()) for n in t_rounds]
    return t_rounds 
def get_ot(map): 
    ot_rounds = map.select('span.mod-ot')
    if(len(ot_rounds) ==0): 
        ot_rounds = [0,0] 
    else: 
        ot_rounds = [ot_rounds[0], ot_rounds[1]]
        ot_rounds = [int(n.get_text().strip()) for n in ot_rounds]
    return ot_rounds
def update_table(map_name, teams, ct_rounds, t_rounds, ot_rounds, winner, map_duration): 
    table = {'Map': '','Map Duration':'', 'Team1':'', 'Team2':'', 
         'Team1 Defense':0, 'Team2 Defense':0, 
         'Team1 Attack':0, 'Team2 Attack':0, 'Team1 OT':0, 'Team2 OT': 0,  
         'Winner':'' }
    table['Map']=map_name
    table['Team1']=teams[0]
    table['Team2']=teams[1]
    table['Team1 Defense']=ct_rounds[0]
    table['Team2 Defense']=ct_rounds[1]
    table['Team1 Attack']=t_rounds[0]
    table['Team2 Attack']=t_rounds[1]
    table['Team1 OT'] = ot_rounds[0]
    table['Team2 OT'] = ot_rounds[1]
    table['Winner']=winner 
    table['Map Duration']=map_duration
    return table 

In [10]:
#varibales 
map_name = '' 
teams = []
ct_rounds = [] 
t_rounds = [] 
ot_rounds = [] 
winner = '' 
map_duration = '' 
match_data = []

In [11]:
for match in match_links: 
    data = requests.get(match)
    soup = BeautifulSoup(data.text)
    maps = get_maps(soup) 
    for map in maps: 
        map_name = map.select_one('.map div span').contents[0].strip()
        map_duration = map.find('div', class_='map-duration').get_text().strip()
        teams = get_teams(map)
        ct_rounds = get_ct(map) 
        t_rounds = get_t(map)
        ot_rounds = get_ot(map)
        winner = determine_winner(ct_rounds, t_rounds, ot_rounds) 
        match_data.append(update_table(map_name, teams, ct_rounds, t_rounds, ot_rounds, winner, map_duration))

In [12]:
df = pd.DataFrame(match_data)
df

,Map,Map Duration,Team1,Team2,Team1 Defense,Team2 Defense,Team1 Attack,Team2 Attack,Team1 OT,Team2 OT,Winner
0,Breeze,55:08,NRG Esports,FURIA,8,6,5,4,0,0,0
1,Ascent,48:14,NRG Esports,FURIA,7,6,6,2,0,0,0
2,Bind,1:06:54,Cloud9,MIBR,5,4,8,7,0,0,0
3,Split,47:29,Cloud9,MIBR,6,7,2,6,0,0,1
4,Lotus,57:07,Cloud9,MIBR,8,7,5,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
198,Ascent,41:31,LOUD,100 Thieves,0,11,1,2,0,0,1
199,Bind,43:12,LOUD,100 Thieves,7,6,6,2,0,0,0
200,Lotus,52:22,LOUD,100 Thieves,7,5,6,5,0,0,0
201,Bind,1:31:34,Sentinels,G2 Esports,0,6,6,7,0,0,1


In [13]:
#cleaning 
#converting categorical data to numeric data for machine learning
#assigning the teams numbers 
df[['Team1','Team2', 'Map']] = df[['Team1','Team2', 'Map']].astype(str)
team_mapping = {"Sentinels": 0, "Cloud9": 1, "G2 Esports": 2, "KRÜ Esports": 3, "NRG Esports": 4, "Leviatán": 5, "LOUD": 6, "100 Thieves": 7, "MIBR":8,"FURIA":9,"Evil Geniuses":10}
df["Team 1 Code"] = df['Team1'].map(team_mapping)
df["Team 2 Code"] = df['Team2'].map(team_mapping)

#note that Haven and Abyss are only to played in stage 2 so they will not be represented in this data 
map_mapping = {"Split": 0, "Icebox": 1, "Sunset": 2, "Bind": 3, "Lotus": 4, "Ascent": 5, "Breeze": 6, "Haven": 7, "Abyss": 8}
df["Map"] = df["Map"].astype(str) 
df["Map Code"] = df["Map"].map(map_mapping) 

In [14]:
#dealing with the rounds 
df[['Team1 Defense','Team1 Attack', 'Team2 Defense', "Team2 Attack",'Team1 OT', 'Team2 OT']] = df[["Team1 Defense",'Team1 Attack', 'Team2 Defense', "Team2 Attack",'Team1 OT', 'Team2 OT']].astype(int)
#total rounds 
df["Total Rounds"] = df[['Team1 Defense','Team1 Attack', 'Team2 Defense', "Team2 Attack",'Team1 OT', 'Team2 OT']].sum(axis=1)

In [15]:
#score for each team
df["Team1 Score"] = df[['Team1 Defense','Team1 Attack','Team1 OT']].sum(axis=1)
df ["Team2 Score"] =df[['Team2 Defense', "Team2 Attack",'Team2 OT']].sum(axis=1)

In [16]:
#round win percentage - rwp
df['Team1 rwp'] = df["Team1 Score"] / df["Total Rounds"] 
df['Team2 rwp'] = df["Team2 Score"] / df["Total Rounds"] 

In [17]:
#percent attack is what percentage of a teams rounds won were from the attacking side 
df['Team1 Attack Percent'] = df['Team1 Attack'] / df['Team1 Score']
df['Team2 Attack Percent'] = df['Team2 Attack'] / df['Team2 Score']

In [18]:
#same for defensive rounds 
df['Team1 Defense Percent'] = df['Team1 Defense'] / df['Team1 Score']
df['Team2 Defense Percent'] = df['Team2 Defense'] / df['Team2 Score']

In [19]:
df

,Map,Map Duration,Team1,Team2,Team1 Defense,Team2 Defense,Team1 Attack,Team2 Attack,Team1 OT,Team2 OT,...,Map Code,Total Rounds,Team1 Score,Team2 Score,Team1 rwp,Team2 rwp,Team1 Attack Percent,Team2 Attack Percent,Team1 Defense Percent,Team2 Defense Percent
0,Breeze,55:08,NRG Esports,FURIA,8,6,5,4,0,0,...,6,23,13,10,0.565217,0.434783,0.384615,0.400000,0.615385,0.600000
1,Ascent,48:14,NRG Esports,FURIA,7,6,6,2,0,0,...,5,21,13,8,0.619048,0.380952,0.461538,0.250000,0.538462,0.750000
2,Bind,1:06:54,Cloud9,MIBR,5,4,8,7,0,0,...,3,24,13,11,0.541667,0.458333,0.615385,0.636364,0.384615,0.363636
3,Split,47:29,Cloud9,MIBR,6,7,2,6,0,0,...,0,21,8,13,0.380952,0.619048,0.250000,0.461538,0.750000,0.538462
4,Lotus,57:07,Cloud9,MIBR,8,7,5,3,0,0,...,4,23,13,10,0.565217,0.434783,0.384615,0.300000,0.615385,0.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Ascent,41:31,LOUD,100 Thieves,0,11,1,2,0,0,...,5,14,1,13,0.071429,0.928571,1.000000,0.153846,0.000000,0.846154
199,Bind,43:12,LOUD,100 Thieves,7,6,6,2,0,0,...,3,21,13,8,0.619048,0.380952,0.461538,0.250000,0.538462,0.750000
200,Lotus,52:22,LOUD,100 Thieves,7,5,6,5,0,0,...,4,23,13,10,0.565217,0.434783,0.461538,0.500000,0.538462,0.500000
201,Bind,1:31:34,Sentinels,G2 Esports,0,6,6,7,0,0,...,3,19,6,13,0.315789,0.684211,1.000000,0.538462,0.000000,0.461538


In [20]:
team_groupings = {}
for team in team_mapping.keys():
    group = df[(df['Team1'] == team) | (df['Team2'] == team)]
    team_groupings[team] = group

In [21]:
team_groupings["Cloud9"]

,Map,Map Duration,Team1,Team2,Team1 Defense,Team2 Defense,Team1 Attack,Team2 Attack,Team1 OT,Team2 OT,...,Map Code,Total Rounds,Team1 Score,Team2 Score,Team1 rwp,Team2 rwp,Team1 Attack Percent,Team2 Attack Percent,Team1 Defense Percent,Team2 Defense Percent
2,Bind,1:06:54,Cloud9,MIBR,5,4,8,7,0,0,...,3,24,13,11,0.541667,0.458333,0.615385,0.636364,0.384615,0.363636
3,Split,47:29,Cloud9,MIBR,6,7,2,6,0,0,...,0,21,8,13,0.380952,0.619048,0.250000,0.461538,0.750000,0.538462
4,Lotus,57:07,Cloud9,MIBR,8,7,5,3,0,0,...,4,23,13,10,0.565217,0.434783,0.384615,0.300000,0.615385,0.700000
18,Sunset,1:00:41,NRG Esports,Cloud9,8,3,5,4,0,0,...,2,20,13,7,0.650000,0.350000,0.384615,0.571429,0.615385,0.428571
19,Ascent,58:34,NRG Esports,Cloud9,7,7,5,5,2,0,...,5,26,14,12,0.538462,0.461538,0.357143,0.416667,0.500000,0.583333
25,Bind,1:16:03,Cloud9,MIBR,5,5,7,7,0,2,...,3,26,12,14,0.461538,0.538462,0.583333,0.500000,0.416667,0.357143
26,Ascent,56:21,Cloud9,MIBR,5,6,4,7,0,0,...,5,22,9,13,0.409091,0.590909,0.444444,0.538462,0.555556,0.461538
51,Icebox,1:10:08,Cloud9,Leviatán,3,4,4,9,0,0,...,1,20,7,13,0.350000,0.650000,0.571429,0.692308,0.428571,0.307692
52,Sunset,46:48,Cloud9,Leviatán,6,1,7,6,0,0,...,2,20,13,7,0.650000,0.350000,0.538462,0.857143,0.461538,0.142857
53,Ascent,1:24:47,Cloud9,Leviatán,5,5,7,7,6,4,...,5,34,18,16,0.529412,0.470588,0.388889,0.437500,0.277778,0.312500


In [22]:
df.to_csv('OneDrive\Documents\match_data7-15.csv', index=False)